<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/Agents/pydantic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install 'pydantic-ai-slim[openai]'

# Step 1: Basic model connector


### Imports and env variables

First we import libs and create env variable for `OpenAIChatModel` to be able to get `OPENROUTER_API_KEY` from env variables

In [6]:
import os

from google.colab import userdata
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openrouter import OpenRouterProvider


OPENROUTER_API_KEY = userdata.get('openrouter')
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

### Initiating model and agent


We create an agent output scheme

In [4]:
class Result(BaseModel):
    answer: str
    confidence: float

> Write system prompt

> create model

> create agent

In [12]:
sys_prompt = "You're a study assistant agent. Your answer should be brief and structured. Avoid emoji and slang."

model = OpenAIChatModel("deepseek/deepseek-chat-v3.1:free",
                        provider=OpenRouterProvider())


agent = Agent[None, Result](model=model,
                            system_prompt=sys_prompt)

### Handling `agent.run` and memory

Simple async run function with an agent decorator. In Pydantic AI the context "lives" while the `agent.run` is being executed (unlike usual LLM request `model.complete` or `model.generate` when it doesn't have access to it's own history/tool use), however between requests there is no memory.

The agent has access to `ctx.deps` - like ram and `ctx.memory_history` - all prompts to the model in current run, after `agent.run` return result the session is closed and the memory is erased.

In [13]:
@agent.run
async def run(ctx: RunContext[None], question: str) -> Result:
    draft = await ctx.llm.complete(question)
    return Result(answer=draft.data, confidence=0.5)

/tmp/ipython-input-1231992350.py:2: RuntimeWarning: coroutine 'AbstractAgent.run' was never awaited
  async def run(ctx: RunContext[None], question: str) -> Result:


### Try it

In [14]:
result = await agent.run("Привет, кто ты?")
print(result)


AgentRunResult(output='Я — ваш помощник-ассистент, предназначенный для ответов на вопросы и помощи в учёбе. Чем могу помочь?')
